In [7]:
from influxdb_client.client.warnings import MissingPivotFunction
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client import InfluxDBClient
import warnings
import pytz
warnings.simplefilter("ignore", MissingPivotFunction)


bucket = "iot"
token = "xlPqJw4yEbmJ2u6vEyD7xVpCqIwcsR3tvIREXQoiYki32rXo43mUdVQ8e5k6YA2TBZQ-FO9Lf-aORU26HTLLcQ=="
org = "polito"
# initializing connect
client = InfluxDBClient(url="http://localhost:8087", token=token, org=org, debug=False)

start_time = '-12h'
stop_time = 'now()'
query_field = 'CPU_usage'
# Write your Flux query:
query = f'from(bucket: "{bucket}")\
          |> range(start: {start_time}, stop: {stop_time}, )\
          |> filter(fn: (r) => r["_field"] == "{query_field}")'       
# initializing query API
query_api = client.query_api()
# querying
result = query_api.query_data_frame(query=str(query)) #DataFrame
# display(result.head())
# disconnect
client.close()

# DataFrame must have the timestamp column as an index for the client.
df = result[['_time', '_value', '_field', '_measurement']]

# Define conversion function
def convert_to_rome_time(row):
    dt = row['_time']
    rome_tz = pytz.timezone('Europe/Rome')
    rome_time = dt.astimezone(rome_tz)
    row['_time'] = rome_time
    return row

# Apply conversion function using apply with axis=1
df = df.apply(convert_to_rome_time, axis=1)
# df.set_index("_time")
# df.head()

df.to_csv('exported.csv',index=False)


In [6]:
df

,_time,_value,_field,_measurement
0,2023-07-06 11:59:50.287101+02:00,30.6,CPU_usage,PC
1,2023-07-06 12:00:00.336081+02:00,29.7,CPU_usage,PC
2,2023-07-06 12:00:10.385321+02:00,29.1,CPU_usage,PC
3,2023-07-06 12:00:20.472675+02:00,29.4,CPU_usage,PC
4,2023-07-06 12:00:30.496953+02:00,30.1,CPU_usage,PC
...,...,...,...,...
1273,2023-07-06 23:58:47.490676+02:00,23.6,CPU_usage,PC
1274,2023-07-06 23:59:02.578819+02:00,28.2,CPU_usage,PC
1275,2023-07-06 23:59:17.659435+02:00,30.3,CPU_usage,PC
1276,2023-07-06 23:59:32.755552+02:00,28.1,CPU_usage,PC
